# YOLO Trainning Code
### Prerequisite Step
**This portion of our code deals with loading the Google Drive on to the local directory of Google Collab**

In [ ]:
# Enable GPU in google collab if not on this portion will flag error.
!nvidia-smi

Mon Aug  1 04:52:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

MessageError: ignored

### 1) Clone the Darknet
**This Porition is essentially responsible for cloning Darknet because Yolo was developed using Darknet framework and runs only on that particular framework**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15460, done.
remote: Total 15460 (delta 0), reused 0 (delta 0), pack-reused 15460
Receiving objects: 100% (15460/15460), 14.07 MiB | 24.55 MiB/s, done.
Resolving deltas: 100% (10387/10387), done.


### 2) Compile Darknet using Nvidia GPU
**This porition is responsible for Compiling Darknet using Nvidia GPU and making slight adjustments to make file**



In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

### 3) Configure Darknet network for training YOLO V3

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
#!mkdir "/mydrive/yolov3"

In [ ]:
!echo "Marker" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/ColabNotebooks/DataMining/YOLO_v3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-07-31 14:46:20--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  18.5MB/s    in 9.5s    

2022-07-31 14:46:30 (16.3 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



### 4) Extract Images

**The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" if any other name used .zip file and folder name is adjustment accordingly on Google Drive**

In [ ]:
!unzip /content/drive/MyDrive/ColabNotebooks/DataMining/YOLO_v3/Images.zip -d data/obj

Archive:  /content/drive/MyDrive/ColabNotebooks/DataMining/YOLO_v3/Images.zip
replace data/obj/IMG_20220731_103157.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/obj/IMG_20220731_103157.jpg  
replace data/obj/IMG_20220731_103157.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace data/obj/IMG_20220731_103157.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/IMG_20220731_103157.txt  
  inflating: data/obj/IMG_20220731_103200.jpg  
  inflating: data/obj/IMG_20220731_103200.txt  
  inflating: data/obj/IMG_20220731_103209.jpg  
  inflating: data/obj/IMG_20220731_103209.txt  
  inflating: data/obj/IMG_20220731_103219.jpg  
  inflating: data/obj/IMG_20220731_103219.txt  
  inflating: data/obj/IMG_20220731_103224.jpg  
  inflating: data/obj/IMG_20220731_103224.txt  
  inflating: data/obj/IMG_20220731_103241.jpg  
  inflating: data/obj/IMG_20220731_103241.txt  
  inflating: data/obj/IMG_20220731_103249.jpg  
  inflating: data/obj/I

In [ ]:
#This part is entirely optional
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.518446 0.727141 0.275608 0.293403

['15', '0.518446', '0.727141', '0.275608', '0.293403']
0 data/obj/IMG_20220731_103504.txt
0 0.518446 0.727141 0.275608 0.293403
0 0.528863 0.473669 0.635851 0.123264

['0', '0.528863', '0.473669', '0.635851', '0.123264']
1 data/obj/IMG_20220731_103934.txt
0 0.528863 0.473669 0.635851 0.123264
0 0.576280 0.565828 0.429905 0.142650

['0', '0.576280', '0.565828', '0.429905', '0.142650']
2 data/obj/IMG_20220731_103824.txt
0 0.576280 0.565828 0.429905 0.142650
0 0.567817 0.491898 0.675130 0.373843

['0', '0.567817', '0.491898', '0.675130', '0.373843']
3 data/obj/IMG_20220731_104017.txt
0 0.567817 0.491898 0.675130 0.373843
15 0.575955 0.443721 0.130208 0.873553

['15', '0.575955', '0.443721', '0.130208', '0.873553']
4 data/obj/IMG_20220731_103445.txt
0 0.575955 0.443721 0.130208 0.873553
0 0.609592 0.516493 0.419705 0.361111

['0', '0.609592', '0.516493', '0.419705', '0.361111']
5 data/obj/IMG_20220731_103726.txt
0 0.609592 0.516493 0.419705 0.361111


In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/IMG_20220731_103724.jpg', 'data/obj/IMG_20220731_103721_01.jpg', 'data/obj/IMG_20220731_103940.jpg', 'data/obj/IMG_20220731_103842.jpg', 'data/obj/IMG_20220731_104134.jpg', 'data/obj/IMG_20220731_103424.jpg', 'data/obj/IMG_20220731_103543.jpg', 'data/obj/IMG_20220731_103528.jpg', 'data/obj/IMG_20220731_103430.jpg', 'data/obj/IMG_20220731_103412.jpg', 'data/obj/IMG_20220731_103555.jpg', 'data/obj/IMG_20220731_103709.jpg', 'data/obj/IMG_20220731_103922.jpg', 'data/obj/IMG_20220731_104115.jpg', 'data/obj/IMG_20220731_103937.jpg', 'data/obj/IMG_20220731_103254.jpg', 'data/obj/IMG_20220731_103540.jpg', 'data/obj/IMG_20220731_103910.jpg', 'data/obj/IMG_20220731_103405.jpg', 'data/obj/IMG_20220731_103512.jpg', 'data/obj/IMG_20220731_103911.jpg', 'data/obj/IMG_20220731_103946.jpg', 'data/obj/IMG_20220731_103906.jpg', 'data/obj/IMG_20220731_104056.jpg', 'data/obj/IMG_20220731_103734_01.jpg', 'data/obj/IMG_20220731_103847.jpg', 'data/obj/IMG_20220731_103334.jpg', 'data/obj/IMG_2022073

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

### 6) Start the training

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

/bin/bash: ./darknet: No such file or directory


In [1]:
print('num_of_processes = 5')
print('process IDs/priorities = 5, 4, 3, 2, 1')
print('coordinator:5')
print('multiple "work:<priority>" messages')
print('failed:5')
print('coordinator:4')
print('multiple "work:<priority>" messages')
print('recovered:5')
print('coordinator:5')
print('multiple "work:<priority>" messages')

num_of_processes = 5
process IDs/priorities = 5, 4, 3, 2, 1
coordinator:5
multiple "work:<priority>" messages
failed:5
coordinator:4
multiple "work:<priority>" messages
recovered:5
coordinator:5
multiple "work:<priority>" messages
